In [2]:
#Packages
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [ ]:
#Get Data
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

In [ ]:
#Get all breaking balls 
bb = all_pitches[all_pitches['TaggedPitchType'].isin(['Curveball', 'Slider','Knuckleball'])]
print(bb.shape)

In [ ]:
#Subset breaking balls into platoon splits
bb_rr = bb[(bb['PitcherThrows'] == 'Right') & (bb['BatterSide'] == 'Right')]
bb_rl = bb[(bb['PitcherThrows'] == 'Right') & (bb['BatterSide'] == 'Left')]
bb_lr = bb[(bb['PitcherThrows'] == 'Left') & (bb['BatterSide'] == 'Right')]
bb_ll = bb[(bb['PitcherThrows'] == 'Left') & (bb['BatterSide'] == 'Left')]

print(bb_rr.shape)
print(bb_rl.shape)
print(bb_lr.shape)
print(bb_ll.shape)

In [ ]:
#Whiffs
bb_rr['whiff'] = np.where(bb_rr['PitchCall']=='StrikeSwinging',1,0)
bb_rl['whiff'] = np.where(bb_rl['PitchCall']=='StrikeSwinging',1,0)
bb_lr['whiff'] = np.where(bb_lr['PitchCall']=='StrikeSwinging',1,0)
bb_ll['whiff'] = np.where(bb_ll['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
#Predictor Variables
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

Righty vs. Righty Split

In [ ]:
bb_rr_X = bb_rr[preds]
bb_rr_y = bb_rr['whiff']

In [ ]:
#Train Model: RHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(bb_rr_X, bb_rr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, predictions, multi_class='ovr')

print(f'ROC AUC: {roc_auc:.4f}')

# Plotting ROC Curve for one class (e.g., class 0)
fpr, tpr, _ = roc_curve(y_test == 1, y_pred_prob)
plt.plot(fpr, tpr, label='Class 0 ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#Auburn Subset
aub_bb_rr = bb_rr[bb_rr['PitcherTeam']=='AUB_TIG']
print(aub_bb_rr.shape)

In [ ]:
# Get unique values
unique_values = bb_rr['Pitcher'].unique()
aub_unique_values = aub_bb_rr['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bb_rr[bb_rr['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bb_rr[bb_rr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rr_bb_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bb_rr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} breaking ball rr stuff+: {np.mean(stuff * 100)}")

    


In [ ]:
#Stuff+ for selected Auburn pitchers Breaking Ball in RHP v RHB split
for pitcher in aub_unique_values:
    aub_rr_bb_stuff(pitcher)

Righty vs. Lefty Split

In [ ]:
bb_rl_X = bb_rl[preds]
bb_rl_y = bb_rl['whiff']

In [ ]:
#Train Model: RHP vs LHB
X_train, X_test, y_train, y_test = train_test_split(bb_rl_X, bb_rl_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bb_rl = bb_rl[bb_rl['PitcherTeam']=='AUB_TIG']
print(aub_bb_rl.shape)

In [ ]:
# Get unique values
unique_values = bb_rl['Pitcher'].unique()
aub_unique_values = aub_bb_rl['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bb_rl[bb_rl['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bb_rl[bb_rl['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rl_bb_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bb_rl_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} breaking ball rl stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Breaking Ball in RHP v LHB split
for pitcher in aub_unique_values:
    aub_rl_bb_stuff(pitcher)

Lefty vs. Righty Split

In [ ]:
bb_lr_X = bb_lr[preds]
bb_lr_y = bb_lr['whiff']

In [ ]:
#Train Model: LHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(bb_lr_X, bb_lr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bb_lr = bb_lr[bb_lr['PitcherTeam']=='AUB_TIG']
print(aub_bb_lr.shape)

In [ ]:
# Get unique values
unique_values = bb_lr['Pitcher'].unique()
aub_unique_values = aub_bb_lr['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bb_lr[bb_lr['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bb_lr[bb_lr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_lr_bb_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bb_lr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} breaking ball lr stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Breaking Ball in LHP v RHB split
for pitcher in aub_unique_values:
    aub_lr_bb_stuff(pitcher)

Lefty vs. Lefty Split

In [ ]:
bb_ll_X = bb_ll[preds]
bb_ll_y = bb_ll['whiff']

In [ ]:
#Train Model: LHP vs LHB
X_train, X_test, y_train, y_test = train_test_split(bb_ll_X, bb_ll_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bb_ll = bb_ll[bb_ll['PitcherTeam']=='AUB_TIG']
print(aub_bb_ll.shape)

In [ ]:
# Get unique values
unique_values = bb_ll['Pitcher'].unique()
aub_unique_values = aub_bb_ll['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bb_ll[bb_ll['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bb_ll[bb_ll['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_ll_bb_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bb_ll_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} breaking ball ll stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Breaking Ball in LHP v LHB split
for pitcher in aub_unique_values:
    aub_ll_bb_stuff(pitcher)